# Secure Federated Learning with Trust Aggregator

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn,optim

W0709 05:05:07.547800 11884 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Telexine\.conda\envs\pysyft2\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0709 05:05:07.565800 11884 deprecation_wrapper.py:119] From C:\Users\Telexine\.conda\envs\pysyft2\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob = sy.VirtualWorker(hook,id="bob")
alice = sy.VirtualWorker(hook,id="alice")
secure_worker = sy.VirtualWorker(hook,id="secure_worker")

bob.clear_objects
alice.clear_objects
secure_worker.clear_objects

<bound method BaseWorker.clear_objects of <VirtualWorker id:secure_worker #objects:0>>

In [3]:
# let them know the ref
bob.add_workers([alice,secure_worker])
alice.add_workers([bob,secure_worker])
secure_worker.add_workers([alice,bob])

W0709 05:05:08.362802 11884 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 05:05:08.363801 11884 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 05:05:08.363801 11884 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0709 05:05:08.364800 11884 base.py:628] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0709 05:05:08.365800 11884 base.py:628] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0709 05:05:08.366801 11884 base.py:628] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [4]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]],requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]],requires_grad=True)

In [5]:
# get pointer to training data on each worker by sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [6]:
alice_data = data[2:].send(alice)
alice_target = target[2:].send(alice)

In [7]:
model = nn.Linear(2,1)

In [11]:
for num_rounds in range(10) :
    bobs_model =  model.copy().send(bob)
    alice_model =  model.copy().send(alice)

    bobs_opt = optim.SGD(params=bobs_model.parameters(),lr=0.1)
    alice_opt = optim.SGD(params=alice_model.parameters(),lr=0.1)
    for i in range(10):
        bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred - bobs_target)**2).sum()
        bobs_loss.backward()
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss

        alice_opt.zero_grad()
        alices_pred = alice_model(alice_data)
        alices_loss = ((alices_pred - alice_target)**2).sum()
        alices_loss.backward()
        alice_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss
    alice_model.move(secure_worker)
    bobs_model.move(secure_worker)
    model.weight.data.set_(((alice_model.weight.data + bobs_model.weight.data ) /2).get())
    model.bias.data.set_(((alice_model.bias.data + bobs_model.bias.data ) /2).get())
    #model.weight.data.set_(((alice_model.weight.data + bobs_model.weight.data ) /2).get())
    #model.bias.data.set_(((alice_model.bias.data + bobs_model.bias.data ) /2).get())
    
    print("Bob:" + str(bobs_loss)+" Alice:"+ str(alices_loss))

RuntimeError: set_storage is not allowed on Tensor created from .data or .detach()

In [12]:
secure_worker._objects

{92470166593: Parameter containing:
 tensor([[0.5494, 0.0550]], requires_grad=True),
 37022156059: Parameter containing:
 tensor([0.4198], requires_grad=True),
 48306987845: Parameter containing:
 tensor([[ 0.0640, -0.0266]], requires_grad=True),
 7456167528: Parameter containing:
 tensor([0.0164], requires_grad=True),
 57325390269: Parameter containing:
 tensor([[0.5494, 0.0550]], requires_grad=True),
 33705474078: Parameter containing:
 tensor([0.4198], requires_grad=True),
 95939079161: Parameter containing:
 tensor([[ 0.0640, -0.0266]], requires_grad=True),
 69831519992: Parameter containing:
 tensor([0.0164], requires_grad=True),
 78662328810: Parameter containing:
 tensor([[0.5494, 0.0550]], requires_grad=True),
 10756964509: Parameter containing:
 tensor([0.4198], requires_grad=True),
 36922542283: Parameter containing:
 tensor([[ 0.0640, -0.0266]], requires_grad=True),
 14823853751: Parameter containing:
 tensor([0.0164], requires_grad=True),
 13157526736: Parameter containing: